# Jupyter Notebook to Analyze EC Standards

2023-9

This notebook is used to compare EC outputs to EC standards for DSM2 planning studies.

The input files are post-processd (daily/monthly) DSS files.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Import modules
import pandas as pd
# import plotly.offline as py
# py.init_notebook_mode(connected=True)
import pydelmod.utilities as pdmu
import pydelmod.nbplot as pdmn

In [3]:
# Read output locations
fpath_output_locations = "../info/DSM2_ec_loc_stds.csv"
df_stations = pd.read_csv(fpath_output_locations, comment='#')
df_stations['ID'] = [x.upper() for x in df_stations['ID']]
station_ids = df_stations['ID'].values
stations_to_read = df_stations['ID'].values

# Build Dataframe for All the Analyzed Data

Only one scenario's Water Year Types (WYT) are used among all scenarios;

i.e. users need to confirm all scenarios share the same WYT; otherwise, WYT tabs and calculation are not suggest to use

Note: different climate change scenarios could have different Water Year types

In [4]:
# Read in scenarios
dir_plan = 'd:/DSM2_Planning_2023/delta/DSM2_v822plan/studies_planning/'
dir2021base = dir_plan+'baseline/'
dir2040alt = dir_plan+'alternative/'

scenarios = [
            {'name': 'baseline',        'fpath': dir2021base+"output/2021ex_EC_p.dss"},
            {'name': 'alternative',     'fpath': dir2040alt+"output/2040alt_EC_p.dss"}
]

# Add a wateryear type column
wyt_c3f2020 = dir_plan+"baseline/timeseries/CALSIM/calsim_2021ex.DSS"
df_wyt2020 = pdmu.read_calsim3_wateryear_types(wyt_c3f2020)

period93 = ['1922-10-1','2015-9-30']

In [5]:
# df_ec = pdmu.prep_df(scenarios,stations_to_read,['EC-MEAN', 'EC-MAX', 'EC-HT-MEAN'],['1DAY', '1MON'],df_wyt2020,period93)
df_ec = pdmu.prep_df(scenarios,stations_to_read,['EC-MEAN'],['1DAY'],df_wyt2020,period93)
df_1day = df_ec[df_ec['interval'] == '1DAY']
df_davg = df_1day[df_1day['variable'] == 'EC-MEAN']

## D1641 Agricultural Standards

Check D1641 details

https://www.waterboards.ca.gov/waterrights/board_decisions/adopted_orders/decisions/d1600_d1649/wrd1641_1999dec29.pdf

Page 182-187, Table 2-3

### D1641_AG West&Interior Delta: 14-day running average of daily mean EC

In [6]:
fpath_d1641_ag = '../info/D1641_AG_wiDelta.csv'
df_reg_ag = pdmu.read_regulations(fpath_d1641_ag, df_wyt2020)
df_reg_ag['value'] = df_reg_ag['value'] * 1000.  # Convert milli to micro
reg_ag_loc = df_reg_ag.location.unique()
df_stations_reg_ag = df_stations[df_stations['ID'].isin(reg_ag_loc)]
# df_stations_reg_ag

In [7]:
# df_1day = df_ec[df_ec['interval'] == '1DAY']
# df_davg = df_1day[df_1day['variable'] == 'EC-MEAN']
df_davg_ag = df_davg[df_davg['station'].isin(reg_ag_loc)]

In [8]:
# Get the time range from the data set
df_reg_ag_ts = pdmu.generate_regulation_timeseries(df_reg_ag, df_davg_ag, freq='D')
df_reg_ag_ts['variable'] = 'EC-MEAN-14DAY'
df_reg_ag_ts['scenario_name'] = 'D1641 AG WI'
# df_reg_ag_ts.hvplot.line()

In [9]:
options = {'yaxis_name': '14-day running average EC (mmhos/cm)', 
           'title': 'EC 14-Day Mean for D1641_AG West&Interior Delta Timelines'}
pdmn.plot_step_w_regulation(df_davg_ag, df_reg_ag_ts, df_stations_reg_ag, options)

    'data': [{'line': {'shape': 'hv'},
              'name': 'baseline',
       …

In [10]:
options = {'yaxis_name': '14-day running average EC - standard (mmhos/cm)', 
           'title': 'EC 14-Day Mean for D1641_AG West&Interior Delta Exceedances'}
pdmn.plot_exceedance_w_regulation(df_davg_ag, df_reg_ag_ts, df_stations_reg_ag, options)

    'data': [{'name': 'baseline',
              'type': 'scatter',
             …

### D1641_AG South Delta: 30-day running average of daily mean EC

In [11]:
fpath_d1641_ag2 = '../info/D1641_AG_sDelta.csv'
df_reg_ag2 = pdmu.read_regulations(fpath_d1641_ag2, df_wyt2020)
df_reg_ag2['value'] = df_reg_ag2['value'] * 1000.  # Convert milli to micro
reg_ag_loc2 = df_reg_ag2.location.unique()
df_stations_reg_ag2 = df_stations[df_stations['ID'].isin(reg_ag_loc2)]
# df_stations_reg_ag2

In [12]:
df_davg_ag2 = df_davg[df_davg['station'].isin(reg_ag_loc2)]

In [13]:
# Get the time range from the data set
df_reg_ag_ts2 = pdmu.generate_regulation_timeseries(df_reg_ag2, df_davg_ag2, freq='D')
df_reg_ag_ts2['variable'] = 'EC-MEAN-30DAY'
df_reg_ag_ts2['scenario_name'] = 'D1641 AG South'
# df_reg_ag_ts2.hvplot.line()

O&M:

start checking on Apr.30, check prev30d-avg; if exceed 700, count 30day, the entire month

May.1, if exceed, count 1day

notebook:

didn't follow the 1st

start checking on Apr.1, count days

In [14]:
options = {'yaxis_name': '30-day running average EC (mmhos/cm)', 
           'title': 'EC 30-Day Mean for D1641_AG South Delta Timelines'}
pdmn.plot_step_w_regulation(df_davg_ag2, df_reg_ag_ts2, df_stations_reg_ag2, options)

    'data': [{'line': {'shape': 'hv'},
              'name': 'baseline',
       …

In [15]:
options = {'yaxis_name': '30-day running average EC - standard (mmhos/cm)', 
           'title': 'EC 30-Day Mean for D1641_AG South Delta Exceedances'}
pdmn.plot_exceedance_w_regulation(df_davg_ag2, df_reg_ag_ts2, df_stations_reg_ag2, options)

    'data': [{'name': 'baseline',
              'type': 'scatter',
             …

### todo, change to monthly

### D1641_AG Export Area: monthly average of daily mean EC

In [16]:
fpath_d1641_ag3 = '../info/D1641_AG_export.csv'
df_reg_ag3 = pdmu.read_regulations(fpath_d1641_ag3, df_wyt2020)
df_reg_ag3['value'] = df_reg_ag3['value'] * 1000.  # Convert milli to micro
reg_ag_loc3 = df_reg_ag3.location.unique()
df_stations_reg_ag3 = df_stations[df_stations['ID'].isin(reg_ag_loc3)]
# df_stations_reg_ag3

In [17]:
df_davg_ag3 = df_davg[df_davg['station'].isin(reg_ag_loc3)]

In [18]:
# Get the time range from the data set
df_reg_ag_ts3 = pdmu.generate_regulation_timeseries(df_reg_ag3, df_davg_ag3, freq='D')
df_reg_ag_ts3['variable'] = 'EC-MEAN-30DAY'
df_reg_ag_ts3['scenario_name'] = 'D1641 AG Export'
# df_reg_ag_ts3.hvplot.line()

In [19]:
options = {'yaxis_name': 'monthly average EC (mmhos/cm)', 
           'title': 'EC Monthly Mean for D1641_AG Export Timelines'}
pdmn.plot_step_w_regulation(df_davg_ag3, df_reg_ag_ts3, df_stations_reg_ag3, options)

    'data': [{'line': {'shape': 'hv'},
              'name': 'baseline',
       …

In [20]:
options = {'yaxis_name': 'monthly average EC - standard (mmhos/cm)', 
           'title': 'EC Monthly Mean for D1641_AG Export Exceedances'}
pdmn.plot_exceedance_w_regulation(df_davg, df_reg_ag_ts3, df_stations_reg_ag3, options)

    'data': [{'name': 'baseline',
              'type': 'scatter',
             …

## D1641 Fish and Wildlife standards

### D1641_FWS San Joaquin River: 14-day running average of daily mean EC

In [21]:
# df_1day = df_ec[df_ec['interval'] == '1DAY']
# df_davg = df_1day[df_1day['variable'] == 'EC-MEAN']

In [22]:
fpath_d1641_fws_sjr = '../info/D1641_FWS_SJR.csv'
df_reg_fws_sjr = pdmu.read_regulations(fpath_d1641_fws_sjr, df_wyt2020)
df_reg_fws_sjr['value'] = df_reg_fws_sjr['value'] * 1000.  # Convert milli to micro
reg_fws_sjr_loc = df_reg_fws_sjr.location.unique()
df_stations_reg_fws_sjr = df_stations[df_stations['ID'].isin(reg_fws_sjr_loc)]
# df_stations_reg_fws_sjr

In [23]:
df_davg_fws_sjr = df_davg[df_davg['station'].isin(reg_fws_sjr_loc)]

In [24]:
# Get the time range from the data set
df_reg_fws_ts_sjr = pdmu.generate_regulation_timeseries(df_reg_fws_sjr, df_davg_fws_sjr, freq='D')
df_reg_fws_ts_sjr['variable'] = 'EC-MEAN-14DAY'
df_reg_fws_ts_sjr['scenario_name'] = 'D1641 FWS SJR'
# df_reg_fws_ts_sjr.hvplot.line()

In [25]:
options = {'yaxis_name': '14-day running average EC (mmhos/cm)', 
           'title': 'EC 14-Day Mean for D1641_FWS San Joaquin River Timelines'}
pdmn.plot_step_w_regulation(df_davg_fws_sjr, df_reg_fws_ts_sjr, df_stations_reg_fws_sjr, options)

    'data': [{'line': {'shape': 'hv'},
              'name': 'baseline',
       …

In [26]:
options = {'yaxis_name': '14-day running average EC - standard (mmhos/cm)', 
           'title': 'EC 14-Day Mean for D1641_FWS San Joaquin River Exceedances'}
pdmn.plot_exceedance_w_regulation(df_davg_fws_sjr, df_reg_fws_ts_sjr, df_stations_reg_fws_sjr, options)

    'data': [{'name': 'baseline',
              'type': 'scatter',
             …

### D1641_FWS Suisun Area: monthly average of daily max EC

In [27]:
df_dmax = pdmu.prep_df(scenarios,stations_to_read,['EC-MAX'],['1DAY'],df_wyt2020,period93)

In [28]:
# df_dmax = df_1day[df_1day['variable'] == 'EC-MAX']

In [29]:
fpath_d1641_fws_suisun = '../info/D1641_FWS_Suisun.csv'
df_reg_fws_suisun = pdmu.read_regulations(fpath_d1641_fws_suisun, df_wyt2020)
df_reg_fws_suisun['value'] = df_reg_fws_suisun['value'] * 1000.  # Convert milli to micro
reg_fws_suisun_loc = df_reg_fws_suisun['location'].unique()
df_stations_reg_fws_suisun = df_stations[df_stations['ID'].isin(reg_fws_suisun_loc)]
# df_stations_reg_fws_suisun

In [30]:
df_dmax_fws_suisun = df_dmax[df_dmax['station'].isin(reg_fws_suisun_loc)]

In [31]:
# Get the time range from the data set
df_reg_fws_suisun_ts = pdmu.generate_regulation_timeseries(df_reg_fws_suisun, df_dmax_fws_suisun, freq='D')
df_reg_fws_suisun_ts['variable'] = 'EC-MAX-MEAN'
df_reg_fws_suisun_ts['scenario_name'] = 'D1641 FWS Suisun'
# df_reg_fws_suisun_ts.hvplot.line()

In [32]:
options = {'yaxis_name': 'EC Monthly Mean of Daily Max (mmhos/cm)', 
           'title': 'EC Monthly Mean of Daily Max for D1641_FWS Suisun Timelines'}
pdmn.plot_step_w_regulation(df_dmax_fws_suisun, df_reg_fws_suisun_ts, df_stations_reg_fws_suisun, options)

    'data': [{'line': {'shape': 'hv'},
              'name': 'baseline',
       …

In [33]:
options = {'yaxis_name': 'Diff in EC scenario-standard (mmhos/cm)', 
           'title': 'EC Monthly Mean of Daily Max for D1641_FWS Suisun Exceedances'} 
pdmn.plot_exceedance_w_regulation(df_dmax_fws_suisun, df_reg_fws_suisun_ts, df_stations_reg_fws_suisun, options)

    'data': [{'name': 'baseline',
              'type': 'scatter',
             …

D1641-FWS Suisun Stations: monthly average of daily average of two high-tide EC

In [34]:
# df_1day = df_ec[df_ec['interval'] == '1DAY']
# df_dht = df_1day[df_1day['variable'] == 'EC-HT-MEAN']

In [35]:
# df_dht

In [36]:
# fpath_d1641_fwss = '../info/D1641_FWS_Standards_Suisun1.csv'
# df_reg_fwss = pdmu.read_regulations(fpath_d1641_fwss, df_wyt)
# df_reg_fwss['value'] = df_reg_fwss['value'] * 1000.  # Convert milli to micro
# df_stations_reg_fwss = df_stations[df_stations['ID'].isin(df_reg_fwss['location'].unique())]
# df_stations_reg_fwss

In [37]:
# options = {'yaxis_name': 'EC Monthly Mean of Daily Average of High-Tide (mmhos/cm)', 'title': 'D1641 FWS Suisun'}
# pdmn.plot_step_w_regulation(df_dht, df_reg_fws_tss, df_stations_reg_fwss, options)

In [38]:
# options = {'yaxis_name': 'Diff in EC scenario-standard (mmhos/cm)', 'title': 'EC Monthly Mean of Daily Average of High-Tide'}
# pdmn.plot_exceedance_w_regulation(df_dht, df_reg_fws_tss, df_stations_reg_fwss, options)

## D1641 Municipal & Industrial standards

### D1641_MI Contra Costa: daily mean Chloride 250 mg/L

In [39]:
fpath_d1641_mi = '../info/D1641_MI_250.csv'
df_reg_mi = pdmu.read_regulations(fpath_d1641_mi, df_wyt2020)
reg_mi_loc = df_reg_mi.location.unique()
df_stations_reg_mi = df_stations[df_stations['ID'].isin(reg_mi_loc)]
# df_stations_reg_mi

In [40]:
reg_mi_loc

array(['ROLD024', 'CHSWP003', 'CHDMC006', 'SLBAR002', 'SLCCH016'],
      dtype=object)

In [41]:
df_davg.station.unique()

array(['CHDMC006', 'CHSWP003', 'OLD_MID', 'RMKL005', 'ROLD024', 'ROLD034',
       'ROLD059', 'RSAC081', 'RSAC092', 'RSAN007', 'RSAN018', 'RSAN032',
       'RSAN037', 'RSAN072', 'RSAN112', 'RSMKL008', 'SLBAR002',
       'SLCBN002', 'SLCCH016', 'SLMZU011', 'SLMZU025', 'SLSUS012'],
      dtype=object)

In [42]:
df_davg_mi = df_davg[df_davg['station'].isin(reg_mi_loc)]

In [43]:
df_davg_cl = df_davg_mi
df_davg_cl['value'] = df_davg_cl.apply(lambda x: max(x['value']*0.15-12.,x['value']*0.285-50),axis=1)
df_davg_cl['variable'] = 'Chloride-MEAN'
# df_davg_cl

D:\DSM2_Planning_2023\delta\DSM2_v822plan\pydelmod_plan\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

D:\DSM2_Planning_2023\delta\DSM2_v822plan\pydelmod_plan\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [44]:
# Get the time range from the data set
df_reg_mi_ts = pdmu.generate_regulation_timeseries(df_reg_mi, df_davg_mi, freq='D')
df_reg_mi_ts['variable'] = 'Chloride-MEAN'
df_reg_mi_ts['scenario_name'] = 'D1641 MI 250'
# df_reg_mi_ts

In [45]:
options = {'yaxis_name': 'Chloride Daily (mg/L)', 
           'title': 'Chloride Daily for D1641 MI 250 Timelines'}
pdmn.plot_step_w_regulation(df_davg_cl, df_reg_mi_ts, df_stations_reg_mi, options)

    'data': [{'line': {'shape': 'hv'},
              'name': 'baseline',
       …

In [46]:
options = {'yaxis_name': 'Chloride Daily -standard (mg/L)', 
           'title': 'Chloride Daily for D1641 MI 250 Exceedances'}
pdmn.plot_exceedance_w_regulation(df_davg_cl, df_reg_mi_ts, df_stations_reg_mi, options)

    'data': [{'name': 'baseline',
              'type': 'scatter',
             …

### D1641_MI Contra Costa: daily mean Chloride 150 mg/L

In [47]:
fpath_d1641_mi2 = '../info/D1641_MI_150.csv'
df_reg_mi2 = pdmu.read_regulations(fpath_d1641_mi2, df_wyt2020)
reg_mi_loc2 = df_reg_mi2.location.unique()
df_stations_reg_mi2 = df_stations[df_stations['ID'].isin(reg_mi_loc2)]
# df_stations_reg_mi2

In [48]:
df_davg_mi2 = df_davg[df_davg['station'].isin(reg_mi_loc2)]

In [49]:
# Get the time range from the data set
df_reg_mi_ts2 = pdmu.generate_regulation_timeseries(df_reg_mi2, df_davg_mi2, freq='D')
df_reg_mi_ts2['variable'] = 'Chloride-MEAN'
df_reg_mi_ts2['scenario_name'] = 'D1641 MI 150'
# df_reg_mi_ts2

In [50]:
df_davg_cl2 = df_davg_mi2
df_davg_cl2['value'] = df_davg_cl2.apply(lambda x: max(x['value']*0.15-12.,x['value']*0.285-50),axis=1)
df_davg_cl2['cl_minus_reg'] = df_davg_cl2.apply(lambda x: x['value'] - 150, axis=1)
df_davg_cl2['cl_meet'] = df_davg_cl2['cl_minus_reg'].map(lambda x: x <= 0)
# df_davg_cl2

D:\DSM2_Planning_2023\delta\DSM2_v822plan\pydelmod_plan\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

D:\DSM2_Planning_2023\delta\DSM2_v822plan\pydelmod_plan\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

D:\DSM2_Planning_2023\delta\DSM2_v822plan\pydelmod_plan\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

In [51]:
# Define an aggregation function to count days when the regulation is met.
# According to D1641:
# Maximum mean daily 150 mg/l Cl−for at least the number of days shown
# during the Calendar Year.
# Must be provided in intervals of not less than two weeks duration.
def sum_only_more_than_14days(series):
    total = 0
    cumsum = 0
    for x in series:
        if x:
            cumsum += 1
        else:
            if cumsum >= 14:
                total += cumsum
            cumsum = 0
    if cumsum >= 14:
        total += cumsum
    return total

In [52]:
df_davg_cl2pl = df_davg_cl2.groupby(['scenario_name', 'year'])['cl_meet'].agg(sum_only_more_than_14days).to_frame()
df_davg_cl2pl['n_records'] = df_davg_cl2.groupby(['scenario_name', 'year'])['year'].count()
df_davg_cl2pl.reset_index(inplace=True)
import calendar
df_davg_cl2pl['days_in_year'] = df_davg_cl2pl['year'].map(lambda x: 366 if calendar.isleap(x) else 365)
df_davg_cl2pl = df_davg_cl2pl[df_davg_cl2pl['n_records'] == df_davg_cl2pl['days_in_year']]
df_davg_cl2pl['time'] = df_davg_cl2pl['year']
df_davg_cl2pl.time = pd.to_datetime(df_davg_cl2pl.time, format='%Y')
df_davg_cl2pl['value'] = df_davg_cl2pl['cl_meet']
df_davg_cl2pl['station'] = 'ROLD024'
df_davg_cl2pl['variable'] = 'Chloride-MEAN'
df_davg_cl2pl = df_davg_cl2pl.merge(df_davg_cl2[["time","scenario_name","sac_yrtype"]], on=["time","scenario_name"], how="left")
# df_davg_cl2pl

Calendar year 1923-2014, only 92years applicable

In [53]:
options = {'yaxis_name': 'Number of days in year under regulation (day)', 
           'title': 'Yearly number of days for D1641 MI 150 Timelines'}
pdmn.plot_step_w_regulation(df_davg_cl2pl, df_reg_mi_ts2, df_stations_reg_mi2, options)

    'data': [{'line': {'shape': 'hv'},
              'name': 'alternative',
    …

In [54]:
options = {'yaxis_name': 'Number of days under regulation - standard (day)', 
           'title': 'Yearly number of days for D1641 MI 150 Exceedances'}
pdmn.plot_exceedance_w_regulation(df_davg_cl2pl, df_reg_mi_ts2, df_stations_reg_mi2, options)

    'data': [{'name': 'alternative',
              'type': 'scatter',
          …